In [1]:

%load_ext autoreload
%autoreload 2

import copy
import pytorch_lightning as pl
import torchinfo
import torchvision.transforms as transforms

from PlantINaturalist2021DataModule import PlantINaturalist2021DataModule
from PlantINaturalist2021FinetuneMobileNetv2 import PlantINaturalist2021FinetuneMobileNetv2, TRANSFORM2

config = {
    "model_name": PlantINaturalist2021FinetuneMobileNetv2.__name__,
    "num_classes": 250,
    "learning_rate": 0.01,
    "lr_decay_epoch_step_size": 5,
    "lr_decay_rate": 0.9,
    "num_trainable_layers": 2,
}

# sanity run
pl.Trainer(max_steps=5).fit(model=PlantINaturalist2021FinetuneMobileNetv2(config), datamodule=PlantINaturalist2021DataModule(transform=TRANSFORM2, context = "retrain"))

model = PlantINaturalist2021FinetuneMobileNetv2(config)

datamodule = PlantINaturalist2021DataModule(transform=TRANSFORM2, context="train", batch_size=32, num_workers=2, pin_memory=True, data_dir="./")

torchinfo.summary(model)

D:\conda\envs\PPC2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
D:\conda\envs\PPC2\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
D:\conda\envs\PPC2\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unles

Sanity Checking DataLoader 0:   0%|                                                              | 0/2 [00:00<?, ?it/s]

D:\conda\envs\PPC2\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


D:\conda\envs\PPC2\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   1%| | 5/470 [00:08<13:44,  1.77s/it, loss=5.83, v_num=22, train_acc_step=0.000, train_precision_step=0.000, 

`Trainer.fit` stopped: `max_steps=5` reached.


Epoch 0:   1%| | 5/470 [00:08<13:51,  1.79s/it, loss=5.83, v_num=22, train_acc_step=0.000, train_precision_step=0.000, 


Layer (type:depth-idx)                                  Param #
PlantINaturalist2021FinetuneMobileNetv2                 --
├─MobileNetV2: 1-1                                      --
│    └─Sequential: 2-1                                  --
│    │    └─Conv2dNormActivation: 3-1                   (928)
│    │    └─InvertedResidual: 3-2                       (896)
│    │    └─InvertedResidual: 3-3                       (5,136)
│    │    └─InvertedResidual: 3-4                       (8,832)
│    │    └─InvertedResidual: 3-5                       (10,000)
│    │    └─InvertedResidual: 3-6                       (14,848)
│    │    └─InvertedResidual: 3-7                       (14,848)
│    │    └─InvertedResidual: 3-8                       (21,056)
│    │    └─InvertedResidual: 3-9                       (54,272)
│    │    └─InvertedResidual: 3-10                      (54,272)
│    │    └─InvertedResidual: 3-11                      (54,272)
│    │    └─InvertedResidual: 3-12                  

In [2]:
import wandb
import torch
from pytorch_lightning.loggers import WandbLogger


run = wandb.init(project='PlantINaturalist', config=config)


wandb: Currently logged in as: valentinmariansavoiu. Use `wandb login --relogin` to force relogin


In [ ]:

wandb_logger = WandbLogger()
trainer = pl.Trainer(benchmark=True, logger=wandb_logger, max_epochs=35, accelerator='gpu', devices=1)

trainer.fit(model=model, datamodule=datamodule)

In [ ]:
torch.save(model.state_dict(), 'model.pth')
artifact = wandb.Artifact('model', type='model')
artifact.add_file('model.pth')
run.log_artifact(artifact)

run.finish()

In [ ]:
next(model.model.classifier.parameters())

In [3]:
import wandb
artifact = run.use_artifact('pasoi0stefan/PlantINaturalist/model:v11', type='model')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [ ]:
model_artifact = PlantINaturalist2021FinetuneMobileNetv2(config)
model_artifact.load_state_dict(torch.load(f"{artifact_dir}/model.pth"))

In [ ]:
#FINETUNE
wandb_logger = WandbLogger()
model_finetune = PlantINaturalist2021FinetuneMobileNetv2(config)
print(next(model_finetune.model.classifier.parameters())[:2])
model_finetune.load_state_dict(torch.load(f"{artifact_dir}/model.pth"))
print(next(model_finetune.model.classifier.parameters())[:2])
model_finetune.learning_rate = 0.001
model_finetune.configure_optimizers()
datamodule_finetune = PlantINaturalist2021DataModule(transform=TRANSFORM2, context="finetune", batch_size=32, num_workers=1, pin_memory=True, data_dir="./")
trainer = pl.Trainer(benchmark=True, logger=wandb_logger, max_epochs = 20, accelerator='gpu', devices=1)
trainer.fit(model=model_finetune, datamodule=datamodule_finetune)
torch.save(model_finetune.state_dict(), 'finetuned_model.pth')
artifact = wandb.Artifact('finetuned_model', type='model')
artifact.add_file('finetuned_model.pth')
run.log_artifact(artifact)
run.finish()

In [ ]:
#FINETUNE 2
wandb_logger = WandbLogger()
model_finetune = PlantINaturalist2021FinetuneMobileNetv2(config)
print(next(model_finetune.model.classifier.parameters())[:2])
model_finetune.load_state_dict(torch.load(f"{artifact_dir}/model.pth"))
print(next(model_finetune.model.classifier.parameters())[:2])
model_finetune.configure_optimizers()
datamodule_finetune = PlantINaturalist2021DataModule(transform=TRANSFORM2, context="retrain", batch_size=32, num_workers=1, pin_memory=True, data_dir="./")
trainer = pl.Trainer(benchmark=True, logger=wandb_logger, max_epochs = 10, accelerator='gpu', devices=1)
trainer.fit(model=model_finetune, datamodule=datamodule_finetune)
torch.save(model_finetune.state_dict(), 'finetuned_model2.pth')
artifact = wandb.Artifact('finetuned_model2', type='model')
artifact.add_file('finetuned_model2.pth')
run.log_artifact(artifact)
run.finish()

In [ ]:
#RETRAIN
wandb.finish()
run = wandb.init()
wandb_logger = WandbLogger()
model_retrain = PlantINaturalist2021FinetuneMobileNetv2(config)
datamodule_retrain = PlantINaturalist2021DataModule(transform=TRANSFORM2, context="retrain", batch_size=32, num_workers=1, pin_memory=True, data_dir="./")
trainer = pl.Trainer(benchmark=True, logger=wandb_logger, max_epochs = 40, accelerator='gpu', devices=1)
trainer.fit(model=model_retrain, datamodule=datamodule_retrain)
torch.save(model_retrain.state_dict(), 'retrained_model.pth')
artifact = wandb.Artifact('retrained_model', type='model')
artifact.add_file('retrained_model.pth')
run.log_artifact(artifact)
run.finish()

D:\conda\envs\PPC2\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | MobileNetV2 | 2.5 M 
--------------------------------------
1.2 M     Trainable params
1.3 M     Non-trainable params
2.5 M     Total params
10.176    Total estimated model params size (MB)


Epoch 0:  83%|▊| 391/470 [00:46<00:09,  8.44it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                  | 0/79 [00:00<?, ?it/s]
Epoch 0:  83%|▊| 392/470 [00:48<00:09,  8.03it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  84%|▊| 393/470 [00:48<00:09,  8.04it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  84%|▊| 394/470 [00:48<00:09,  8.04it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  84%|▊| 395/470 [00:49<00:09,  8.05it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  84%|▊| 396/470 [00:49<00:09,  8.06it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  84%|▊| 397/470 [00:49<00:09,  8.06it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0: 

Epoch 0:  97%|▉| 455/470 [00:54<00:01,  8.36it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  97%|▉| 456/470 [00:54<00:01,  8.36it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  97%|▉| 457/470 [00:54<00:01,  8.37it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  97%|▉| 458/470 [00:54<00:01,  8.37it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  98%|▉| 459/470 [00:54<00:01,  8.37it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  98%|▉| 460/470 [00:54<00:01,  8.38it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  98%|▉| 461/470 [00:54<00:01,  8.38it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  98%|▉| 462/470 [00:55<00:00,  8.39it/s, loss=5.35, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 0:  99%|▉| 463/470 [00:55<00:00,  

Epoch 1:  93%|▉| 439/470 [00:55<00:03,  7.86it/s, loss=4.93, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 1:  94%|▉| 440/470 [00:55<00:03,  7.86it/s, loss=4.93, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 1:  94%|▉| 441/470 [00:56<00:03,  7.87it/s, loss=4.93, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 1:  94%|▉| 442/470 [00:56<00:03,  7.87it/s, loss=4.93, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 1:  94%|▉| 443/470 [00:56<00:03,  7.88it/s, loss=4.93, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 1:  94%|▉| 444/470 [00:56<00:03,  7.88it/s, loss=4.93, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 1:  95%|▉| 445/470 [00:56<00:03,  7.89it/s, loss=4.93, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 1:  95%|▉| 446/470 [00:56<00:03,  7.89it/s, loss=4.93, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 1:  95%|▉| 447/470 [00:56<00:02,  

Epoch 2:  90%|▉| 423/470 [00:52<00:05,  7.98it/s, loss=4.36, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 2:  90%|▉| 424/470 [00:53<00:05,  7.99it/s, loss=4.36, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 2:  90%|▉| 425/470 [00:53<00:05,  8.00it/s, loss=4.36, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 2:  91%|▉| 426/470 [00:53<00:05,  8.01it/s, loss=4.36, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 2:  91%|▉| 427/470 [00:53<00:05,  8.01it/s, loss=4.36, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 2:  91%|▉| 428/470 [00:53<00:05,  8.02it/s, loss=4.36, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 2:  91%|▉| 429/470 [00:53<00:05,  8.03it/s, loss=4.36, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 2:  91%|▉| 430/470 [00:53<00:04,  8.03it/s, loss=4.36, v_num=2y5t, train_acc_step=0.050, train_precision_step=0.0
Epoch 2:  92%|▉| 431/470 [00:53<00:04,  

...

Epoch 29:  96%|▉| 453/470 [00:55<00:02,  8.12it/s, loss=1.94, v_num=2y5t, train_acc_step=0.550, train_precision_step=0.
Epoch 29:  97%|▉| 454/470 [00:55<00:01,  8.12it/s, loss=1.94, v_num=2y5t, train_acc_step=0.550, train_precision_step=0.
Epoch 29:  97%|▉| 455/470 [00:55<00:01,  8.13it/s, loss=1.94, v_num=2y5t, train_acc_step=0.550, train_precision_step=0.
Epoch 29:  97%|▉| 456/470 [00:56<00:01,  8.14it/s, loss=1.94, v_num=2y5t, train_acc_step=0.550, train_precision_step=0.
Epoch 29:  97%|▉| 457/470 [00:56<00:01,  8.14it/s, loss=1.94, v_num=2y5t, train_acc_step=0.550, train_precision_step=0.
Epoch 29:  97%|▉| 458/470 [00:56<00:01,  8.15it/s, loss=1.94, v_num=2y5t, train_acc_step=0.550, train_precision_step=0.
Epoch 29:  98%|▉| 459/470 [00:56<00:01,  8.15it/s, loss=1.94, v_num=2y5t, train_acc_step=0.550, train_precision_step=0.
Epoch 29:  98%|▉| 460/470 [00:56<00:01,  8.16it/s, loss=1.94, v_num=2y5t, train_acc_step=0.550, train_precision_step=0.
Epoch 29:  98%|▉| 461/470 [00:56<00:01, 

Epoch 30:  93%|▉| 437/470 [00:50<00:03,  8.58it/s, loss=1.65, v_num=2y5t, train_acc_step=0.750, train_precision_step=0.
Epoch 30:  93%|▉| 438/470 [00:51<00:03,  8.58it/s, loss=1.65, v_num=2y5t, train_acc_step=0.750, train_precision_step=0.
Epoch 30:  93%|▉| 439/470 [00:51<00:03,  8.59it/s, loss=1.65, v_num=2y5t, train_acc_step=0.750, train_precision_step=0.
Epoch 30:  94%|▉| 440/470 [00:51<00:03,  8.59it/s, loss=1.65, v_num=2y5t, train_acc_step=0.750, train_precision_step=0.
Epoch 30:  94%|▉| 441/470 [00:51<00:03,  8.60it/s, loss=1.65, v_num=2y5t, train_acc_step=0.750, train_precision_step=0.
Epoch 30:  94%|▉| 442/470 [00:51<00:03,  8.61it/s, loss=1.65, v_num=2y5t, train_acc_step=0.750, train_precision_step=0.
Epoch 30:  94%|▉| 443/470 [00:51<00:03,  8.61it/s, loss=1.65, v_num=2y5t, train_acc_step=0.750, train_precision_step=0.
Epoch 30:  94%|▉| 444/470 [00:51<00:03,  8.62it/s, loss=1.65, v_num=2y5t, train_acc_step=0.750, train_precision_step=0.
Epoch 30:  95%|▉| 445/470 [00:51<00:02, 

Epoch 31:  90%|▉| 421/470 [00:54<00:06,  7.73it/s, loss=1.79, v_num=2y5t, train_acc_step=0.500, train_precision_step=0.
Epoch 31:  90%|▉| 422/470 [00:54<00:06,  7.74it/s, loss=1.79, v_num=2y5t, train_acc_step=0.500, train_precision_step=0.
Epoch 31:  90%|▉| 423/470 [00:54<00:06,  7.74it/s, loss=1.79, v_num=2y5t, train_acc_step=0.500, train_precision_step=0.
Epoch 31:  90%|▉| 424/470 [00:54<00:05,  7.75it/s, loss=1.79, v_num=2y5t, train_acc_step=0.500, train_precision_step=0.
Epoch 31:  90%|▉| 425/470 [00:54<00:05,  7.75it/s, loss=1.79, v_num=2y5t, train_acc_step=0.500, train_precision_step=0.
Epoch 31:  91%|▉| 426/470 [00:54<00:05,  7.76it/s, loss=1.79, v_num=2y5t, train_acc_step=0.500, train_precision_step=0.
Epoch 31:  91%|▉| 427/470 [00:55<00:05,  7.76it/s, loss=1.79, v_num=2y5t, train_acc_step=0.500, train_precision_step=0.
Epoch 31:  91%|▉| 428/470 [00:55<00:05,  7.77it/s, loss=1.79, v_num=2y5t, train_acc_step=0.500, train_precision_step=0.
Epoch 31:  91%|▉| 429/470 [00:55<00:05, 

Epoch 32:  86%|▊| 405/470 [00:51<00:08,  7.90it/s, loss=1.72, v_num=2y5t, train_acc_step=0.350, train_precision_step=0.
Epoch 32:  86%|▊| 406/470 [00:51<00:08,  7.91it/s, loss=1.72, v_num=2y5t, train_acc_step=0.350, train_precision_step=0.
Epoch 32:  87%|▊| 407/470 [00:51<00:07,  7.92it/s, loss=1.72, v_num=2y5t, train_acc_step=0.350, train_precision_step=0.
Epoch 32:  87%|▊| 408/470 [00:51<00:07,  7.92it/s, loss=1.72, v_num=2y5t, train_acc_step=0.350, train_precision_step=0.
Epoch 32:  87%|▊| 409/470 [00:51<00:07,  7.93it/s, loss=1.72, v_num=2y5t, train_acc_step=0.350, train_precision_step=0.
Epoch 32:  87%|▊| 410/470 [00:51<00:07,  7.94it/s, loss=1.72, v_num=2y5t, train_acc_step=0.350, train_precision_step=0.
Epoch 32:  87%|▊| 411/470 [00:51<00:07,  7.94it/s, loss=1.72, v_num=2y5t, train_acc_step=0.350, train_precision_step=0.
Epoch 32:  88%|▉| 412/470 [00:51<00:07,  7.95it/s, loss=1.72, v_num=2y5t, train_acc_step=0.350, train_precision_step=0.
Epoch 32:  88%|▉| 413/470 [00:51<00:07, 

Epoch 33:  83%|▊| 391/470 [00:45<00:09,  8.68it/s, loss=1.61, v_num=2y5t, train_acc_step=0.500, train_precision_step=0.
Validation: 0it [00:00, ?it/s]

In [ ]:
next(model_artifact.model.classifier.parameters())